# 🛫Gate Agent Assistant 

We know gate agents are expert reservation system users, but what if a flight attendant or fleet service agent needed to stand-in momentarily?  Or how about trainees? Can we make getting started with the reservation system easier?  This example shows how to make a simple Gate Agent assistant bot using mocked up reservation systems data.  

Instructions for getting started with this Ultralight Backpackers AI trek are on the [GitHub page's README](../README.md). 

# Ultralight gear 📃list for 🤖AI-driven gate agent assistant

- 1 [Semantic Plugin Chat](./plugins-sk/Reservations/Chat/skprompt.txt) to give the bot a Gate Agent assistant persona.
- 1 [Semantic Plugin TestData](./plugins-sk/Reservations/TestData/skprompt.txt) to create test data.
- 1 [Native Plugin GateAgentPlugin](../plugins/GateAgentPlugin.cs) to fullfill reservation system commands and create test itineraries.  (We are not connecting to a real reservation system, it's all pretend)



> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Gather gear for our trek 🔦🥾⛺

When running the following cell, if asked to "select your kernel" (note this will be referring to the Jupyter notebook's kernel and not Semantic Kernel) then choose `.NET Interactive` from the available menu options.

In [ ]:
#!import ../config/Settings.cs
#!import ../plugins/GateAgentPlugin.cs
#r "nuget: Microsoft.SemanticKernel, *-*"

using Microsoft.SemanticKernel.Planners;
using Microsoft.SemanticKernel;
using System.IO;

// Grab the locally stored credentials from the settings.json file. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, unused1, unused2, unused3) = Settings.LoadFromFile();
var builder = new KernelBuilder();
builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
IKernel kernel = builder.Build();

var planner = new ActionPlanner(kernel);
var semanticPlugins = kernel.ImportSemanticFunctionsFromDirectory("plugins-sk", "Reservations");
var myContext = new ContextVariables();

# Use AI to create some 🧪test PNR records
Use genAI to create test data for a 🔁genAI Bot. 

In [ ]:
var sample = ReservationSystemSkill.getSampleData();
var myResult = await kernel.RunAsync(sample,semanticPlugins["TestData"]);
//Console.WriteLine(myResult.GetValue<string>());
var gateAgentPlugin = new ReservationSystemSkill(myResult.GetValue<string>());
gateAgentPlugin.Context = myContext;
var functions = kernel.ImportFunctions(gateAgentPlugin);

# 🥾Hike the ➰loop as many times as you like
This is a very easy trail and only has a few features.
- find ininerary or find PNR, for example, ask it to "Find PNR for Andrew Skurka"
- change a seat, ex: "Change seat to 23D", it'll figure things out from context.
- Show the PNR, ex: "show PNR details as a table with column headings"
- Ask it questions about the itinerary, ex: "what is the fligth duration"

Some of the capabilities are from the plugin, others are native capabilities of the language model.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var botPrompt = "AI: Hello, standing by to help at Gate C17.";
Console.WriteLine(botPrompt);
var history = $"{botPrompt}\n";
const int numberOfRounds = 40;
myContext.Set("history", history); 

for(var i = 0; i < numberOfRounds; i++) {
    try {
        // get input from the user and set the context variable
        var input = await InteractiveKernel.GetInputAsync($"{botPrompt}");
        myContext.Set("input", (botPrompt.Contains("(clarification)")) ?  myContext["last_input"] + $" {input}" : input);     
        string functionName = "Chat"; 
        try { // action planner is unstable for some input
            var plan = await planner.CreatePlanAsync(myContext["input"]);
             if (plan.HasNextStep && ReservationSystemSkill.hasParameters(plan)) {
                functionName = "GateAgent";
                await plan.InvokeAsync(kernel);
            }
        } catch {  }
        var botResponse = (await kernel.RunAsync(myContext,semanticPlugins[functionName])).GetValue<string>();
 
        // tack onto the history 👇 what's come back from the model
        myContext.Set("history", history += $"Me: {input}\nAI:{botResponse}\n"); // remember it
        Console.WriteLine($"Me: {input}\nAI:{botResponse}\n"); // show it 
 
        // prepare to "prompt" the user with the bot's response
        botPrompt = $"AI: {botResponse}";
        myContext.Set("last_input",input);
    } catch {        
        // if the user hits "Escape" we end the chat early
        Console.WriteLine("AI: Thanks for the wonderful chat!");
        break;
    }
}